In [1]:
import pandas as pd
import numpy as np

In [2]:
colley_df = pd.read_csv('../../data/final_data/colley/matches.csv', parse_dates = ['Date'])

In [3]:
colley_df

,League,Season,Date,Goals_Home,Team_Home,ColleyRating_Home,Game#_Home,Goals_Away,Team_Away,ColleyRating_Away,Game#_Away
0,Ligue1,2017-2018,2017-08-04,3,Monaco,0.500000,0,2,Toulouse,0.500000,0
1,Ligue1,2017-2018,2017-08-05,2,Paris S-G,0.500000,0,0,Amiens,0.500000,0
2,Ligue1,2017-2018,2017-08-05,1,Montpellier,0.500000,0,0,Caen,0.500000,0
3,Ligue1,2017-2018,2017-08-05,4,Lyon,0.500000,0,0,Strasbourg,0.500000,0
4,Ligue1,2017-2018,2017-08-05,1,Saint-Étienne,0.500000,0,0,Nice,0.500000,0
...,...,...,...,...,...,...,...,...,...,...,...
6969,LaLiga,2020-2021,2021-04-25,2,Sevilla,0.693349,32,1,Granada,0.473506,31
6970,SerieA,2020-2021,2021-04-25,3,Cagliari,0.342528,32,2,Roma,0.615254,32
6971,Bundesliga,2020-2021,2021-04-25,5,M'Gladbach,0.527877,30,0,Arminia,0.386034,30
6972,Bundesliga,2020-2021,2021-04-25,2,RB Leipzig,0.683608,30,0,Stuttgart,0.484511,30


In [4]:
from sklearn.metrics import brier_score_loss

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [5]:
colley_df['Win_Home'] = 1 * (colley_df['Goals_Home'] > colley_df['Goals_Away'])

In [7]:
cols_for_pred = [
    'I(ColleyRating_Home - ColleyRating_Away)',
]

glm_str = "Win_Home ~ " + '+'.join(cols_for_pred)
print(glm_str)

result_glm = smf.glm(glm_str, 
             data = colley_df[colley_df['Season'].isin(['2017-2018', '2018-2019'])], 
             family = sm.families.Binomial()
).fit()
result_glm.summary()

Win_Home ~ I(ColleyRating_Home - ColleyRating_Away)


<class 'statsmodels.iolib.summary.Summary'>
"""
                 Generalized Linear Model Regression Results                  
==============================================================================
Dep. Variable:               Win_Home   No. Observations:                 3652
Model:                            GLM   Df Residuals:                     3650
Model Family:                Binomial   Df Model:                            1
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2321.4
Date:                Wed, 12 May 2021   Deviance:                       4642.8
Time:                        18:35:41   Pearson chi2:                 3.66e+03
No. Iterations:                     4                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Intercept                                   -0.2036      0.035     -5.803      0.000      -0.272      -0.135
I(ColleyRating_Home - ColleyRating_Away)     3.3736      0.186     18.138      0.000       3.009       3.738
============================================================================================================
"""

In [8]:
print("Validation Brier Score: ", brier_score_loss(colley_df[colley_df['Season'].isin(['2019-2020'])]['Win_Home'], result_glm.predict(colley_df[colley_df['Season'].isin(['2019-2020'])])))


Validation Brier Score:  0.2283591334537125


In [9]:
from sklearn.metrics import accuracy_score
print("Validation Accuracy Score: ", accuracy_score(colley_df[colley_df['Season'].isin(['2019-2020'])]['Win_Home'], 1 * (result_glm.predict(colley_df[colley_df['Season'].isin(['2019-2020'])]) > 0.5)))


Validation Accuracy Score:  0.6255072463768115
